Install Required Packages

In [2]:
!pip install langchain -qU
!pip install langchain-chroma -qU
!pip install langchain_community -qU
!pip install google-generativeai --no-deps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 353.3 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.2 MB/s eta

Create SQLite Database and Tables

In [3]:
import sqlite3

In [4]:
connection = sqlite3.connect("school.db")

In [5]:
cursor = connection.cursor()

In [6]:

cursor.execute("""
CREATE TABLE IF NOT EXISTS teachers (
    teacher_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS subjects (
    subject_id INTEGER PRIMARY KEY AUTOINCREMENT,
    subject_name TEXT NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS teaches (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    teacher_id INTEGER,
    subject_id INTEGER,
    FOREIGN KEY (teacher_id) REFERENCES teachers(teacher_id),
    FOREIGN KEY (subject_id) REFERENCES subjects(subject_id)
)
""")

connection.commit()

Insert Sample Data into Tables

In [7]:
teachers = [
    ("Kamal", 42),
    ("Saman", 29),
    ("Pawan", 34)
]
cursor.executemany("INSERT INTO teachers (name, age) VALUES (?, ?)", teachers)

subjects = [
    ("Mathematics",),
    ("Science",),
    ("History",),
    ("English",)
]
cursor.executemany("INSERT INTO subjects (subject_name) VALUES (?)", subjects)

teaches = [
    (1, 1),  # Kamal teaches Mathematics
    (1, 2),  # Kamal teaches Science
    (2, 3),  # Saman teaches History
    (3, 4),  # Pawan teaches English
]
cursor.executemany("INSERT INTO teaches (teacher_id, subject_id) VALUES (?, ?)", teaches)

connection.commit()


In [8]:
cursor.execute("SELECT * FROM teachers")
teachers = cursor.fetchall()
print(teachers)


[(1, 'Kamal', 42), (2, 'Saman', 29), (3, 'Pawan', 34)]


In [9]:
cursor.execute("""SELECT t.name
FROM teachers t
JOIN teaches te ON t.teacher_id = te.teacher_id
JOIN subjects s ON te.subject_id = s.subject_id
WHERE s.subject_name = 'Mathematics';""")
teachers = cursor.fetchall()
print(teachers)

[('Kamal',)]


Initialize LangChain SQL Database

In [10]:
from langchain_community.utilities.sql_database import SQLDatabase

In [11]:
db = SQLDatabase.from_uri(f"sqlite:///school.db")

In [12]:
print(db.get_usable_table_names())

['subjects', 'teachers', 'teaches']


In [13]:
print(db.table_info)


CREATE TABLE subjects (
	subject_id INTEGER, 
	subject_name TEXT NOT NULL, 
	PRIMARY KEY (subject_id)
)

/*
3 rows from subjects table:
subject_id	subject_name
1	Mathematics
2	Science
3	History
*/


CREATE TABLE teachers (
	teacher_id INTEGER, 
	name TEXT NOT NULL, 
	age INTEGER NOT NULL, 
	PRIMARY KEY (teacher_id)
)

/*
3 rows from teachers table:
teacher_id	name	age
1	Kamal	42
2	Saman	29
3	Pawan	34
*/


CREATE TABLE teaches (
	id INTEGER, 
	teacher_id INTEGER, 
	subject_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(teacher_id) REFERENCES teachers (teacher_id), 
	FOREIGN KEY(subject_id) REFERENCES subjects (subject_id)
)

/*
3 rows from teaches table:
id	teacher_id	subject_id
1	1	1
2	1	2
3	2	3
*/


Initialize LLM

In [14]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 474.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.0 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [15]:
import os
from google.colab import userdata


In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Set OpenAI API key
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

# Initialize the ChatOpenAI model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0
)

Create SQL Generation Chain

In [17]:
from langchain.chains import create_sql_query_chain

query_generate = create_sql_query_chain(llm, db)

Setup SQL Execution Tool

In [18]:
from langchain_community.tools import QuerySQLDatabaseTool

query_execute = QuerySQLDatabaseTool(db=db)

In [21]:
# Step 1: Generate SQL using LLM
query = query_generate.invoke({"question": "Details of all teachers"})

# Step 2: Extract only the SQL part from the LLM output
if "SQLQuery:" in query:
    query = query.split("SQLQuery:")[-1].strip()

In [22]:
print(query)

SELECT "teacher_id", "name", "age" FROM teachers LIMIT 5


In [23]:
query_execute.invoke(query)

"[(1, 'Kamal', 42), (2, 'Saman', 29), (3, 'Pawan', 34)]"

In [25]:
query = query_generate.invoke({"question": "Which teachers are assigned to teach Mathematics?"})
# Step 2: Extract only the SQL part from the LLM output
if "SQLQuery:" in query:
    query = query.split("SQLQuery:")[-1].strip()
print(query)

SELECT T1."name" FROM teachers AS T1 INNER JOIN teaches AS T2 ON T1."teacher_id" = T2."teacher_id" INNER JOIN subjects AS T3 ON T2."subject_id" = T3."subject_id" WHERE T3."subject_name" = 'Mathematics'


In [26]:
query_execute.invoke(query)


"[('Kamal',)]"

Create Answer Generator Chain

In [31]:
from langchain_core.runnables import RunnableLambda

# Function to extract pure SQL from LLM output
def extract_sql(output: str) -> str:
    if "SQLQuery:" in output:
        return output.split("SQLQuery:")[-1].strip()
    return output.strip()

# Add to pipeline as a Runnable
extract_sql_node = RunnableLambda(extract_sql)


In [32]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough


# Final answer prompt template (already defined correctly)
answer_prompt = PromptTemplate.from_template(
    """Given a user question, the generated SQL query, and its result, write a clear and natural answer to the question.

User Question: {question}
SQL Query: {query}
SQL Result: {result}

Final Answer:"""
)

final_answer = answer_prompt | llm | StrOutputParser()

# The corrected chain
chain = (
    RunnablePassthrough.assign(
        query=query_generate | extract_sql_node  # Clean SQL!
    ).assign(
        result=itemgetter("query") | query_execute
    )
    | final_answer
)



In [33]:
chain.invoke({"question": "Details of all teachers"})

'There are 3 teachers in total. Their details are:\n*   Kamal, age 42, with teacher ID 1.\n*   Saman, age 29, with teacher ID 2.\n*   Pawan, age 34, with teacher ID 3.'

In [34]:
chain.invoke({"question": "Which teachers are assigned to teach Mathematics?"})

'The teacher assigned to teach Mathematics is Kamal.'

In [37]:
chain.invoke({"question": "Details of all subjects"})

'Here are the details of the subjects:\n*   **Mathematics** (ID: 1)\n*   **Science** (ID: 2)\n*   **History** (ID: 3)\n*   **English** (ID: 4)'

In [39]:
chain.invoke({"question": "Which teachers are assigned to teach history?"})

'The teacher assigned to teach history is Saman.'